# 2D model without preprocessing

## Colab integration

In [ ]:
RUNNING_IN_COLAB = False

if RUNNING_IN_COLAB:
    REPO_URL = 'https://github.com/nicomem/imed-project.git'
    REPO_DIR = 'imed-project'
    DATA_URL = 'https://drive.google.com/uc?id=1onHHWIhkhN5xYMit0rhhtVXlJrAlzCit'
    
    from pathlib import Path

    %cd /content

    # Download the repository
    if not Path(REPO_DIR).is_dir():
        !git clone {REPO_URL} {REPO_DIR} --depth=1
    
    %cd {REPO_DIR}

    # Install requirements
    !pip install -r requirements.txt | grep -v 'Requirement already satisfied'
    !pip install gdown | grep -v 'Requirement already satisfied'
    
    import gdown
    if not Path('data.zip').is_file():
        gdown.download(DATA_URL, 'data.zip', quiet=False)
    
    if not Path('data').is_dir():
        !unzip data.zip
    
    %cd notebooks
    !ls

## Imports

In [ ]:
# 3rd-party imports
import numpy as np
import nibabel as nib
import tensorflow as tf
import tensorflow.keras as k
import matplotlib.pyplot as plt
import keras_unet

## Get dataset & split train/test

In [ ]:
from utils.load_data import get_dataset, SlicesSequence, CachedSlicesSequence

train_nib, val_nib = get_dataset('../data', verbose=True)

In [ ]:
train_nib.keys()

In [ ]:
[len(v) for v in train_nib.values()]

In [ ]:
[len(v) for v in val_nib.values()]

## Load train & analyze

In [ ]:
batch_size = 32
shuffle = True
target_height = 256
target_width = 256

train_seq_uncached = SlicesSequence(train_nib,
                                    target_height, target_width, 
                                    batch_size, shuffle)
train_seq = CachedSlicesSequence(train_seq_uncached, batch_size, shuffle)
len(train_seq)

In [ ]:
# Different number of slices & X/Y dimensions for inputs & targets
x,y = train_seq[0]
print(x.shape)
print(y.shape)
print(x[0,...,0].shape, x[0,...,1].shape, y[0].shape)

print('---')

x,y = train_seq[-1]
print(x.shape)
print(y.shape)
print(x[0,...,0].shape, x[0,...,1].shape, y[0].shape)

In [ ]:
plt.figure(figsize=(9, 6))
i_data = 15

x,y = train_seq[0]
plt.subplot(2, 3, 1); plt.imshow(x[i_data,...,0])
plt.subplot(2, 3, 2); plt.imshow(x[i_data,...,1])
plt.subplot(2, 3, 3); plt.imshow(y[i_data])

x,y = train_seq[-1]
plt.subplot(2, 3, 4); plt.imshow(x[i_data,...,0])
plt.subplot(2, 3, 5); plt.imshow(x[i_data,...,1])
plt.subplot(2, 3, 6); plt.imshow(y[i_data])

## Prepare the model

In [ ]:
from keras_unet.models import custom_unet

input_shape = (target_height, target_width, 2)

model = custom_unet(
    input_shape,
    num_classes=1,
    use_batch_norm=False,
    filters=16,
    num_layers=4,
    dropout=0.3,
    output_activation='sigmoid'
)
model.summary()

## Train the model

In [ ]:
val_seq_uncached = SlicesSequence(val_nib,
                                  target_height, target_width,
                                  batch_size, shuffle)
val_seq = CachedSlicesSequence(val_seq_uncached, batch_size, shuffle)
len(val_seq)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)

In [ ]:
history = model.fit(train_seq, epochs=10, validation_data=val_seq)

## Check the results

In [ ]:
def rolling_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

ravg_w = 1

plt.plot(rolling_average(history['loss'], ravg_w), label=f'{name} (train)')

plt.plot(rolling_average(history['val_loss'], ravg_w), label=f'{name} (train)')

plt.legend()
plt.show()